# 3層の深層学習で分類

## データの取得
Rにあるspamデータ

In [5]:
import csv
def open_with_python_csv(filename):
    data = []
    with open(filename, 'r') as filename:
        reader = csv.reader(filename)
        # ヘッダ行は特別扱い
        header = next(reader)
        # 中身
        for row in reader:
            data.append(row)
    return header, data

header, data = open_with_python_csv("spam.csv")

In [12]:
print(data[1])
print(data[1][0:57])

['0.21', ' 0.28', '0.50', ' 0.00', ' 0.14', '0.28', '0.21', ' 0.07', '0.00', ' 0.94', '0.21', '0.79', '0.65', ' 0.21', '0.14', ' 0.14', '0.07', '0.28', ' 3.47', ' 0.00', ' 1.59', ' 0.00', '0.43', ' 0.43', ' 0.00', ' 0.00', ' 0.00', '0.00', ' 0.00', '0.00', ' 0.00', '0.00', ' 0.00', '0.00', ' 0.00', '0.00', '0.07', '0.00', ' 0.00', '0.00', '0.00', ' 0.00', '0.00', ' 0.00', ' 0.00', ' 0.00', '0.00', ' 0.00', '0.000', '0.132', '0.000', ' 0.372', '0.180', ' 0.048', '   5.114', ' 101', ' 1028', 'spam']
['0.21', ' 0.28', '0.50', ' 0.00', ' 0.14', '0.28', '0.21', ' 0.07', '0.00', ' 0.94', '0.21', '0.79', '0.65', ' 0.21', '0.14', ' 0.14', '0.07', '0.28', ' 3.47', ' 0.00', ' 1.59', ' 0.00', '0.43', ' 0.43', ' 0.00', ' 0.00', ' 0.00', '0.00', ' 0.00', '0.00', ' 0.00', '0.00', ' 0.00', '0.00', ' 0.00', '0.00', '0.07', '0.00', ' 0.00', '0.00', '0.00', ' 0.00', '0.00', ' 0.00', ' 0.00', ' 0.00', '0.00', ' 0.00', '0.000', '0.132', '0.000', ' 0.372', '0.180', ' 0.048', '   5.114', ' 101', ' 1028']


In [21]:
import numpy as np
spam = np.array(data[:][:])

In [29]:
#説明変数
spam_x = spam[:, 0:57]
#被説明変数
spam_y = spam[:, 57]

In [65]:
#数値配列に変換
spam_x2 = spam_x.astype(np.float32)
#0,1に変換
spam_y2 = np.zeros([spam_y.shape[0], 2])
for i in range(4601):
    if(spam_y[i] == "spam"):
        spam_y2[i, 0] = 1
    else:
        spam_y2[i, 1] = 1

## 訓練データとテストデータに分ける

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(spam_x2, spam_y2, train_size=0.5, random_state=1)

## 深層学習

### 3層のNN

In [114]:
import tensorflow as tf

session = tf.InteractiveSession()

#プレースホルダーの定義
#これに実際のデータ入れる
x = tf.placeholder(tf.float32, [None, 57])
y_ = tf.placeholder(tf.float32, [None, 2])
keep_prob = tf.placeholder(tf.float32)   #dropoutする確率を変数

#重みとバイアス
#tf.random_normal : 正規分布からランダムな値を出力
w1 = tf.Variable(tf.random_normal([57, 10]))
b1 = tf.Variable(tf.zeros([10]))

#中間層入力
u1 = tf.matmul(x, w1) + b1
y1 = tf.sigmoid(u1)
#y1 = tf.nn.relu(u1)

#3層目出力
#tf.truncated_normal : 正規分布からランダムな値を出力
#w2 = tf.Variable(tf.truncated_normal([10, 4]))
w2 = tf.Variable(tf.truncated_normal([10, 2]))
#b2 = tf.Variable(tf.zeros([4]))
b2 = tf.Variable(tf.zeros([2]))
u2 = tf.matmul(y1, w2) + b2
#y2 = tf.sigmoid(u2)
#ドロップアウト層なし
y2 = tf.nn.softmax(u2)

#ドロップアウト層
#h_fc_drop = tf.nn.dropout(y2, keep_prob)   

#出力を定義
#w3 = tf.Variable(tf.truncated_normal([4, 2]))
#b3 = tf.Variable(tf.zeros([2]))
#u3 = tf.matmul(h_fc_drop, w3) + b3
#y3 = tf.nn.softmax(u3)

#ドロップアウトなし
y = y2
#ドロップアウトあり
#y = y3

#交差エントロピーによる誤差関数
cross_entoropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
#optimizer = tf.train.GradientDescentOptimizer(0.1)
optimizer = tf.train.AdamOptimizer(1e-3)
train_step = optimizer.minimize(cross_entoropy)

# モデルの評価 
predict_step = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))#予想した数字があってるか
accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))#正解率

#代入変数の定義
#別にいらんな
def set_feed(xx, yy, prob):
    return {x: xx, y_: yy, keep_prob: prob}

#テスト用のfeed
test_fd = set_feed(X_val, y_val, 1)

#訓練
init = tf.initialize_all_variables()
session.run(init)

for step in range(10000):
    #訓練データから適当に100個取得
    sampleindices = random.sample(range(X_train.shape[0]), k=100)
    batch_x = X_train[sampleindices, :]
    batch_y = y_train[sampleindices, :]
    fd = set_feed(batch_x, batch_y , 0.5)
    
    _, loss = session.run([train_step, cross_entoropy], feed_dict=fd)
        
    #1000ステップごとに損失と正解率を出力
    if step % 1000 == 0:
        acc = session.run(accuracy_step, feed_dict=test_fd)
        print("step=", step, "loss=", loss, "acc=", acc)
    
    #train_step.run(fd)
    #train_step.run({x: X_train[sampleindices, :], y_: y_train[sampleindices, :], keep_prob:0.5})
#train_step.run({x: X_train, t: y_train})

#テスト
acc = session.run(accuracy_step, feed_dict=test_fd)
print("正解率=", acc)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(test_fd) * 100, '%')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step= 0 loss= 1.1152 acc= 0.590178
step= 1000 loss= 0.388824 acc= 0.878314
step= 2000 loss= 0.214691 acc= 0.894828
step= 3000 loss= 0.193604 acc= 0.916123
step= 4000 loss= 0.19046 acc= 0.924815
step= 5000 loss= 0.112461 acc= 0.931769
step= 6000 loss= 0.118473 acc= 0.933942
step= 7000 loss= 0.153105 acc= 0.929596
step= 8000 loss= 0.13627 acc= 0.932638
step= 9000 loss= 0.105176 acc= 0.931769
正解率= 0.93003
93.0030405521 %


In [61]:
X_train.shape

(2300, 57)

In [62]:
y_train.shape

(2300,)